Import libary

In [7]:
#ImpoRT the library to be useb
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import math
import pandas as pd

-----

access variables outside the script

In [9]:
%store -r mtx
%store -r dist
%store -r roi
%store -r newcameramtx
%store -r calibratePixel

---

Create a frame for show image has been process

In [8]:
#function for show image
def display_img(img,cmap=None):
    #create size image width 12in and hight 10in
    fig = plt.figure(figsize=(32,30))
    #plot 1, row 1, colom 1
    ax = fig.add_subplot(111)
    #show image
    ax.imshow(img,cmap)

---

Search point to create line

In [30]:
# function for search intersection point line with circle
def pointSearch(xInner, yInner, r, beta):
    # empty arr for x axis dan y axis
    x = []
    y = []

    r = r + 10

    # degree direction Opencv is CW
    # determine the calculation by determining the angle, where the angles of 90 and 270 are not allowed
    if beta != 90 and beta != 270:
        betaInv = -beta

        # calculate roots
        a = math.pow(1/math.cos(math.radians(betaInv)), 2)
        b = -2*xInner*math.pow(1/math.cos(math.radians(betaInv)), 2)
        c = math.pow(xInner, 2)*math.pow(1/math.cos(math.radians(betaInv)), 2) - math.pow(r, 2)

        # roots for x axis
        x1 = (-b + math.sqrt(math.pow(b, 2) - 4*a*c))/(2*a)
        x2 = (-b - math.sqrt(math.pow(b, 2) - 4*a*c))/(2*a)
        x.append(x1)
        x.append(x2)

        # roots for y axis
        y1 = math.tan(math.radians(betaInv))*x1 - math.tan(math.radians(betaInv))*xInner + yInner
        y2 = math.tan(math.radians(betaInv))*x2 - math.tan(math.radians(betaInv))*xInner + yInner
        y.append(y1)
        y.append(y2)

        # combines each coordinate into a tuple
        pointTest = list(zip(x, y))

        # determines the limit of the x value based on the angle
        # limits for angles over 270 and less than 90
        if beta > 270 or beta < 90:
            for xTest, yTest in pointTest:
                if xTest > xInner:
                    pass
                else:
                    # remove roots that don't fit the constraints
                    x.remove(xTest)
                    y.remove(yTest)
        # limits for angles over 90 and less than 270
        else:
            for xTest, yTest in pointTest:
                if xTest < xInner:
                    pass
                else:
                    # remove roots that don't fit the constraints
                    x.remove(xTest)
                    y.remove(yTest)
    # for angles 90 and 270
    else:
        # angles 90 and 270 do not follow the equation of the line so as to create a perpendicular line x = xInner
        x.append(xInner)

        # calculate roots
        a = 1
        b = -2*yInner
        c = math.pow(yInner, 2) - math.pow(r, 2)

        # roots of y axis
        y1 = (-b + math.sqrt(math.pow(b, 2) - 4*a*c))/(2*a)
        y2 = (-b - math.sqrt(math.pow(b, 2) - 4*a*c))/(2*a)
        y.append(y1)
        y.append(y2)

        # point requirements at an angle of 90
        if beta == 90:
            for yTest in y:
                if yTest < yInner:
                    pass
                else:
                    y.remove(yTest)
        # point requirements at an angle of 270
        else:
            for yTest in y:
                if yTest > yInner:
                    pass
                else:
                    y.remove(yTest)

    # round up the x and y coordinates
    x = math.ceil(x[0])
    y = math.ceil(y[0])

    return x, y

---

function to measure the thickness of roi

In [36]:
def thickMeasure(xInner, yInner, xPoint, yPoint, roi=None):
    # create line by angle
    line = cv2.line(mask.copy(),(xCenter, yCenter),(xPoint, yPoint),(255, 255, 255), 1)
    line = cv2.cvtColor(line, cv2.COLOR_BGR2GRAY)

    # removes the non-subjects from the logical result set and
    roiThick = cv2.bitwise_and(line, roi)

    # get delta x and delta y
    indexWhite = np.where(roiThick == 255)
    deltaX = indexWhite[1].max() - indexWhite[1].min()
    deltaY = indexWhite[0].max() - indexWhite[0].min()

    # roi thickness in pixels
    thickPixel = math.sqrt(math.pow(deltaX, 2)+math.pow(deltaY, 2))

    # roll thickness in mm
    thick = thickPixel * constCalibrate

    return roiThick, thickPixel, thick

---
function to draw weld points

In [37]:
def drawDotWeld(roiCap, lengthThick, weldPoint, angle):
    searchWhite = np.where(roiCap == 255)
    a = searchWhite[1].max() - searchWhite[1].min()
    b = searchWhite[0].max() - searchWhite[0].min()

    d = (weldPoint*a)/lengthThick
    e = (weldPoint*b)/lengthThick

    # find the starting point of the line
    if angle > 0 and angle < 90:
        xIntial = searchWhite[1].min()
        yIntial = searchWhite[0].max()

        xDot = d + xIntial
        yDot = yIntial - e
    elif angle >= 90 and angle <= 180:
        xIntial = searchWhite[1].max()
        yIntial = searchWhite[0].max()

        xDot = xIntial - d
        yDot = yIntial - e
    elif angle > 180 and angle < 270:
        xIntial = searchWhite[1].max()
        yIntial = searchWhite[0].min()

        xDot = xIntial - d
        yDot = e + yIntial
    elif (angle >= 270 and angle < 360) or angle == 0:
        xIntial = searchWhite[1].min()
        yIntial = searchWhite[0].min()

        xDot = d + xIntial
        yDot = e + yIntial

    xDot = math.ceil(xDot)
    yDot = math.ceil(yDot)

    return xDot, yDot

---

location image

In [71]:
path = 'Image Irrdiation Container\\A1.jpg'

---

remove lens distortion on image

In [72]:
# call image form path
imgRaw = cv2.imread(path)
#undistort
img = cv2.undistort(imgRaw, mtx, dist, None, newcameramtx)
x, y, w, h = roi
img = img[y:y+h, x:x+w]

---

image processing

In [73]:
#change color fomar from BGR to Grayscale
imgGray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
#copy image like img
imgDraw = img.copy()
imgDrawLine = img.copy()
imgDrawDot = img.copy()

In [75]:
#reduce noise by gaussian blur kernel size 5x5
imgBlur = cv2.GaussianBlur(imgGray,(5,5),0)

In [77]:
#edge detection
imgEdge = cv2.Canny(imgBlur,50,100)

In [79]:
#create empty array to draw circel or line
mask = np.zeros(img.shape,np.uint8)

In [80]:
__,width,__ = img.shape[:]

---
Find Inner, Middle, and Outter Circle each part iradiation capsule

Inner Circle Detection

In [81]:
detectInnerCircle = cv2.HoughCircles(imgEdge, cv2.HOUGH_GRADIENT ,1.5, width/2, param1=60, param2=50, minRadius=90, maxRadius=130)

In [82]:
if detectInnerCircle is not None:
    # Convert the circle parameters a, b and r to integers.
    detectInnerCircle = np.uint16(np.around(detectInnerCircle))
    for pt in detectInnerCircle[0, :]:
        #a and b is coordinates midle point circle, a --> x, b --> y, r is radius circle
        a, b, r = pt[0], pt[1], pt[2]
        # Draw the circumference of the circle.
        drawInner = cv2.circle(imgDraw, (a, b), r, (255, 0, 0), 2)
        drawFullInnerCircle = cv2.circle(mask.copy(), (a, b), r, (255, 255, 255), -1)
    print('berhasil')
else:
    print('tidak terdeteksi')

berhasil


---
Middel Circle Detection

In [83]:
detectMidCircle = cv2.HoughCircles(imgEdge, cv2.HOUGH_GRADIENT ,1.5, width/2, param1=60, param2=50, minRadius=150, maxRadius=170)

In [84]:
if detectMidCircle is not None:
    # Convert the circle parameters a, b and r to integers.
    detectMidCircle = np.uint16(np.around(detectMidCircle))
    for pt in detectMidCircle[0, :]:
        #a and b is coordinates midle point circle, a --> x, b --> y, r is radius circle
        a, b, r = pt[0], pt[1], pt[2]
        # Draw the circumference of the circle.
        drawMid = cv2.circle(imgDraw, (a, b), r, (0, 255, 0), 2)
        drawFullMidCircle = cv2.circle(mask.copy(), (a, b), r, (255, 255, 255), -1)
    print('berhasil')
else:
    print('tidak terdeteksi')

berhasil


---
Outter Circle Detection

In [85]:
detectOutterCircle = cv2.HoughCircles(imgEdge, cv2.HOUGH_GRADIENT ,1.5, width/2, param1=60, param2=50, minRadius=180, maxRadius=230)

In [86]:
if detectOutterCircle is not None:
    # Convert the circle parameters a, b and r to integers.
    detectOutterCircle = np.uint16(np.around(detectOutterCircle))
    for pt in detectOutterCircle[0, :]:
        #a and b is coordinates midle point circle, a --> x, b --> y, r is radius circle
        a, b, r = pt[0], pt[1], pt[2]
        # Draw the circumference of the circle.
        drawOutter = cv2.circle(imgDraw, (a, b), r, (0, 0, 255), 2)
        drawFullOutterCircle = cv2.circle(mask.copy(), (a, b), r, (255, 255, 255), -1)
    print('berhasil')
else:
    print('tidak terdeteksi')

berhasil


---
Do operational logic to get roi part of iradiation capsule

get cap iradiation capsule

In [98]:
#remove object undisire and leaving roi cap
areaCap = cv2.bitwise_xor(drawFullMidCircle,drawFullInnerCircle)
areaCap = cv2.cvtColor(areaCap,cv2.COLOR_BGR2GRAY)

In [99]:
#get detail cap on roi cap do operational logical and
cap = cv2.bitwise_and(imgBlur,areaCap)

In [100]:
#do threshold to get more detail and create binary image 
# __,thresCap = cv2.threshold(cap,250,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
thresCap = cv2.adaptiveThreshold(cap, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV, 5, 2)

get tube iradiation capsule

In [102]:
areaTube = cv2.bitwise_xor(drawFullMidCircle,drawFullOutterCircle)
areaTube = cv2.cvtColor(areaTube,cv2.COLOR_BGR2GRAY)

In [103]:
tube = cv2.bitwise_and(imgBlur,areaTube)

In [104]:
# __,thresTube = cv2.threshold(tube,250,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
thresTube = cv2.adaptiveThreshold(tube, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV, 5, 2)

---

Calculate thickness by length line given logical operator and with roi

In [31]:
xCenter,yCenter,__ = detectInnerCircle[0][0]

In [32]:
__,__,rOutter = detectOutterCircle[0][0]

In [34]:
constCalibrate = calibratePixel

In [38]:
dataThick = pd.DataFrame(columns=['sudut (derajat)','tutup kapsul (piksel)','tutup kapsul (mm)','tabung kapsul (piksel)','tabung kapsul (mm)','titik las (piksel)','titik las (mm)','arah titik las dari celah kapsul (piksel)','arah titik las dari celah kapsul (mm)'])

In [39]:
for degree in range(0,361,1):
    #mencari titik singgung dengan lingakaran terluar
    x, y = pointSearch(xCenter,yCenter,rOutter,degree)
    
    drawLine = cv2.line(imgDrawLine,(xCenter,yCenter),(x,y),(255,0,0),2)
    
    #mengukur ketebalan tutup kapsul
    roiLineCap, capThickPix, capThick = thickMeasure(xCenter,yCenter,x,y,roi=thresCap)
    
    #mengukur ketebalan tabung kapsul
    __, tubeThickPix, tubeThick = thickMeasure(xCenter,yCenter,x,y,roi=thresTube)
    
    #menhitung titik las
    pointWeldPix = (capThickPix + tubeThickPix)/2
    pointWeld =  pointWeldPix*constCalibrate
    
    #menghitung arah titik las dari celah kapsul
    #jika positif --> mengarah ke tabung
    #jika negatif --> mengarah ke tutup
    dirWeldPix = pointWeldPix - capThickPix
    dirWeld = dirWeldPix*constCalibrate
    
    #gambar titik las
    xDotWeld,yDotWeld = drawDotWeld(roiLineCap,capThickPix,pointWeldPix,degree)
    
    dataThick.loc[len(dataThick)] = [degree,capThickPix,capThick,tubeThickPix,tubeThick,pointWeldPix,pointWeld,dirWeldPix,dirWeld]
    
    #titik las
    drawDot = cv2.circle(imgDrawDot, (xDotWeld,yDotWeld), 1, (255, 0, 255), 2)

In [ ]:
#save thickness data to excel
dataThick.to_excel('Output\\A1\\A1.xlsx')

In [ ]:
display_img(drawDot)